```{contents}
```
## **Agent Memory in LangGraph**

**Agent Memory** in LangGraph is the collection of mechanisms that allow an agent to **store, retrieve, update, and reason over information across time and executions**.
It enables **learning, long-horizon reasoning, personalization, and continuity**—all of which are impossible in stateless pipelines.

---

### **1. Why Agent Memory Is Necessary**

Without memory, an agent behaves like a **single-turn function**.
With memory, it becomes a **persistent intelligent system**.

| Capability             | Without Memory | With Memory  |
| ---------------------- | -------------- | ------------ |
| Context retention      | No             | Yes          |
| Multi-step planning    | Limited        | Long-horizon |
| Personalization        | None           | Persistent   |
| Learning from mistakes | No             | Yes          |
| Autonomous behavior    | Fragile        | Robust       |

---

### **2. Memory Layers in LangGraph**

LangGraph implements memory as a **multi-layer architecture**.

```
┌───────────────────────────────┐
│   Long-Term Memory (Vector DB)│
├───────────────────────────────┤
│   Short-Term Memory (State)   │
├───────────────────────────────┤
│   Working Memory (Messages)   │
└───────────────────────────────┘
```

| Layer             | Purpose                   | Lifetime    |
| ----------------- | ------------------------- | ----------- |
| Working Memory    | Current reasoning context | One step    |
| Short-Term Memory | Graph state               | One run     |
| Long-Term Memory  | Persistent knowledge      | Across runs |

---

### **3. Memory Types**

| Type                  | Implementation             |
| --------------------- | -------------------------- |
| **Working Memory**    | `messages` in state        |
| **Short-Term Memory** | State fields in graph      |
| **Episodic Memory**   | Checkpoints                |
| **Semantic Memory**   | Vector store               |
| **Procedural Memory** | Graph structure + policies |
| **Tool Memory**       | Tool outputs               |
| **User Memory**       | Profile stored in DB       |

---

### **4. Implementing Memory: Core Pattern**

#### **State Definition**

```python
class AgentState(TypedDict):
    messages: list
    user_profile: dict
    task_memory: list
```

This state becomes the agent’s **short-term memory**.

---

### **5. Long-Term Memory with Vector Store**

```python
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

memory_store = FAISS.from_texts([], OpenAIEmbeddings())
```

#### **Writing to Memory**

```python
def store_memory(state):
    memory_store.add_texts([state["messages"][-1]["content"]])
    return {}
```

#### **Retrieving from Memory**

```python
def retrieve_memory(state):
    docs = memory_store.similarity_search(state["messages"][-1]["content"], k=3)
    return {"long_term_context": [d.page_content for d in docs]}
```

---

### **6. Memory-Augmented Agent Loop**

```
User Input
   ↓
Retrieve Memory
   ↓
Reason (LLM)
   ↓
Act (Tools)
   ↓
Store Experience
   ↓
Update State
   ↓
Repeat
```

---

### **7. Checkpointing: Episodic Memory**

LangGraph supports persistent execution memory:

```python
from langgraph.checkpoint import SqliteSaver

checkpointer = SqliteSaver("agent.db")
graph = builder.compile(checkpointer=checkpointer)
```

This allows:

* Resume after crash
* Replay execution
* Inspect historical states

---

### **8. Agent Memory in Multi-Agent Systems**

| Memory Scope  | Usage                    |
| ------------- | ------------------------ |
| Agent-local   | Individual learning      |
| Shared memory | Team coordination        |
| Global memory | Organizational knowledge |

Agents communicate through **shared memory fields** in state.

---

### **9. Production Memory Architecture**

```
LangGraph
   |
State Store (Redis)
   |
Vector DB (Pinecone / FAISS / Weaviate)
   |
Relational DB (Postgres)
```

| Component        | Stores                |
| ---------------- | --------------------- |
| Redis            | Short-term state      |
| Vector DB        | Semantic memory       |
| Postgres         | User & session memory |
| Checkpoint store | Execution history     |

---

### **10. Design Best Practices**

* Separate **working memory** from **long-term memory**
* Limit context size with retrieval + summarization
* Use memory decay & pruning
* Encrypt sensitive memory
* Version memory schema
* Add human review for memory writes

---

### **11. Why Memory Makes Agents Intelligent**

| Property        | Explanation           |
| --------------- | --------------------- |
| Continuity      | Understands past      |
| Learning        | Improves over time    |
| Personalization | Adapts to user        |
| Autonomy        | Handles long goals    |
| Resilience      | Recovers from failure |

---

### **Mental Model**

> **Memory is the agent’s mind.
> LangGraph provides the brain structure that manages it.**


### Demonstration

In [5]:
# ============================
# AGENT MEMORY DEMONSTRATION
# ============================

from typing import TypedDict, List
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.messages import HumanMessage, AIMessage

# ---------- Long-Term Memory (Vector DB) ----------
embeddings = OpenAIEmbeddings()
# Initialize with a dummy document to avoid empty list error
vector_store = FAISS.from_texts(["System initialized"], embeddings)

# ---------- Agent State ----------
class AgentState(TypedDict):
    messages: List
    long_term_context: List[str]

# ---------- Nodes ----------

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

def retrieve_memory(state: AgentState):
    """Retrieve relevant memories from vector store."""
    if not state["messages"]:
        return {"long_term_context": []}
    
    # Get the last user message
    last_message = state["messages"][-1]
    query = last_message.content if hasattr(last_message, 'content') else str(last_message)
    
    # Search for relevant memories
    docs = vector_store.similarity_search(query, k=2)
    context = [d.page_content for d in docs] if docs else []
    
    print(f"Retrieved {len(context)} memories")
    return {"long_term_context": context}

def agent_reason(state: AgentState):
    """Agent reasons using current input and long-term memory."""
    context = "\n".join(state["long_term_context"]) if state["long_term_context"] else "No prior memory"
    
    last_message = state["messages"][-1]
    user_query = last_message.content if hasattr(last_message, 'content') else str(last_message)
    
    prompt = f"""You are a helpful assistant with memory.

Relevant past memories:
{context}

User: {user_query}

Respond naturally using both current input and past memories when relevant."""

    response = llm.invoke(prompt)
    return {"messages": state["messages"] + [AIMessage(content=response.content)]}

def store_memory(state: AgentState):
    """Store important information to long-term memory."""
    if len(state["messages"]) >= 2:
        # Store both user message and AI response as context
        last_user_msg = state["messages"][-2]
        last_ai_msg = state["messages"][-1]
        
        user_content = last_user_msg.content if hasattr(last_user_msg, 'content') else str(last_user_msg)
        ai_content = last_ai_msg.content if hasattr(last_ai_msg, 'content') else str(last_ai_msg)
        
        memory_entry = f"User: {user_content}\nAssistant: {ai_content}"
        vector_store.add_texts([memory_entry])
        print(f"Stored memory: {memory_entry[:100]}...")
    
    return {}

# ---------- Build Graph ----------
builder = StateGraph(AgentState)

builder.add_node("retrieve", retrieve_memory)
builder.add_node("reason", agent_reason)
builder.add_node("store", store_memory)

builder.set_entry_point("retrieve")
builder.add_edge("retrieve", "reason")
builder.add_edge("reason", "store")
builder.add_edge("store", END)

# ---------- Checkpointing (Episodic Memory) ----------
checkpointer = MemorySaver()
graph = builder.compile(checkpointer=checkpointer)

# ---------- Run ----------
print("=== Agent Memory Demo ===\n")

thread_config = {"configurable": {"thread_id": "demo-user-123"}}

# First interaction - store information
print("--- Interaction 1 ---")
result1 = graph.invoke(
    {"messages": [HumanMessage(content="My name is Sanjeev and I like machine learning.")], 
     "long_term_context": []},
    thread_config
)
print(f"Agent: {result1['messages'][-1].content}\n")

# Second interaction - recall information
print("--- Interaction 2 ---")
result2 = graph.invoke(
    {"messages": [HumanMessage(content="What do I like?")], 
     "long_term_context": []},
    thread_config
)
print(f"Agent: {result2['messages'][-1].content}\n")

# Third interaction - recall name
print("--- Interaction 3 ---")
result3 = graph.invoke(
    {"messages": [HumanMessage(content="What's my name?")], 
     "long_term_context": []},
    thread_config
)
print(f"Agent: {result3['messages'][-1].content}")

=== Agent Memory Demo ===

--- Interaction 1 ---
Retrieved 1 memories
Stored memory: User: My name is Sanjeev and I like machine learning.
Assistant: Hi Sanjeev! It's great to see you a...
Agent: Hi Sanjeev! It's great to see you again. Since you like machine learning, is there a particular area or project you're currently working on or interested in?

--- Interaction 2 ---
Retrieved 2 memories
Stored memory: User: What do I like?
Assistant: You like machine learning! Is there a specific aspect of it that yo...
Agent: You like machine learning! Is there a specific aspect of it that you're currently exploring or any projects you're working on?

--- Interaction 3 ---
Retrieved 2 memories
Stored memory: User: What's my name?
Assistant: Your name is Sanjeev! It's nice to chat with you again. How have yo...
Agent: Your name is Sanjeev! It's nice to chat with you again. How have you been?
